# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [1]:
import pandas as pd
import chart_studio.plotly as py
import cufflinks as cf
from ipywidgets import interact
import ipywidgets as widgets
import plotly.express as px
import dash
from dash import dcc, html, Input, Output

import plotly.graph_objects as go

from IPython.display import display


cf.go_offline()

/Users/sergej/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
data = pd.read_excel('../data/Online Retail.xlsx')

In [3]:
data.drop_duplicates(inplace=True)

In [4]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

data.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom


## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [5]:
data_april_2011 = data[
    (data['InvoiceDate'].dt.year == 2011) & 
    (data['InvoiceDate'].dt.month == 4) & 
    (data['Country'] != 'United Kingdom')
]

agg_data_april_2011 = data_april_2011.groupby('Country').agg({'Quantity': 'sum', 'Revenue': 'sum'}).reset_index()
agg_data_april_2011.head()

,Country,Quantity,Revenue
0,Australia,224,421.60
1,Austria,308,584.78
2,Belgium,1170,1788.48
3,Brazil,356,1143.60
4,Channel Islands,96,243.00


In [6]:
def plot_bar_chart():
    fig = px.bar(agg_data_april_2011, 
                 x="Country", 
                 y=["Revenue", "Quantity"], 
                 barmode="group", 
                 title="Total Quantity and Revenue by Country (April 2011, Excluding UK)")
    fig.show()
plot_bar_chart()

## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [7]:
data_france_2011 = data[
    (data['InvoiceDate'] >= '2011-01-01') & 
    (data['InvoiceDate'] <= '2011-05-31') & 
    (data['Country'] == 'France')
]

agg_data_france_2011 = data_france_2011.groupby(data_france_2011['InvoiceDate'].dt.normalize()).agg({'Quantity': 'sum', 'Revenue': 'sum'}).reset_index()
agg_data_france_2011.sample(10)

,InvoiceDate,Quantity,Revenue
25,2011-02-21,333,728.82
63,2011-05-16,278,672.62
9,2011-01-19,383,698.75
18,2011-02-08,214,165.86
5,2011-01-11,978,1332.64
10,2011-01-20,702,1408.68
46,2011-04-04,293,513.55
20,2011-02-10,452,285.56
21,2011-02-15,96,219.60
52,2011-04-26,84,160.54


In [8]:
fig = px.line(
    agg_data_france_2011,
    x='InvoiceDate',
    y=['Quantity', 'Revenue'],
    title='France Sales: Quantity and Revenue from January 1 to May 31, 2011',
    labels={'InvoiceDate': 'Date', 'value': 'Sales'}
)

fig.show()

## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [9]:
data_party = data[data["Description"] == "PARTY BUNTING"]

agg_data_party = data_party.groupby("Country").agg(
    avg_quantity=("Quantity", "mean"),
    avg_unit_price=("UnitPrice", "mean")
).reset_index()

fig = px.scatter(
    agg_data_party,
    x="avg_quantity", 
    y="avg_unit_price", 
    color="Country",              
    size="avg_quantity",        
    hover_name="Country",        
    title="Relationship Between Average Quantity and Average Unit Price for 'PARTY BUNTING'"
)

fig.update_traces(marker=dict(opacity=0.8, line=dict(width=1, color='DarkSlateGrey')))
fig.show()

## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

In [10]:
selected_countries = ['EIRE', 'Germany', 'France', 'Netherlands']
data_selected = data[data['Country'].isin(selected_countries)]

agg_invoices = data_selected.groupby(['Country', 'InvoiceNo'])['Quantity'].sum().reset_index()
agg_invoices.rename(columns={'Quantity': 'TotalQuantity'}, inplace=True)

fig = px.histogram(
    agg_invoices,
    x='TotalQuantity',
    facet_col='Country',
    facet_col_wrap=2,  
    title='Distribution of Total Quantity per Invoice for EIRE, Germany, France, and Netherlands',
    labels={'TotalQuantity': 'Quantity per Invoice'},
    nbins=50 
)

fig.update_layout(height=600)  
fig.show()


## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [11]:
product_list = ['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']

country_list = ['EIRE', 'Germany', 'France', 'Netherlands']

In [12]:
product_list = ['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']

country_list = ['EIRE', 'Germany', 'France', 'Netherlands']

data_filtered = data[
    data['Description'].isin(product_list) &
    data['Country'].isin(country_list)
]
agg_data = data_filtered.groupby(['Description', 'Country'])['Revenue'].sum().reset_index()

fig = px.bar(
    agg_data,
    x='Description',     
    y='Revenue',       
    color='Country',    
    barmode='group',     
    title='Revenue by Country for Selected Products'
)

fig.show()

## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [13]:
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
data['Day'] = pd.DatetimeIndex(data['InvoiceDate']).day
uk = data[data['Country']=='United Kingdom']

In [14]:
uk = data[data['Country'] == 'United Kingdom']

# Initialize Dash app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("UK Sales: Quantity Sold by Day"),

    html.Label("Select Year:"),
    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': str(y), 'value': y} for y in sorted(uk['Year'].unique())],
        value=uk['Year'].min()
    ),

    html.Label("Select Month:"),
    dcc.Dropdown(id='month-dropdown'),

    dcc.Graph(id='line-chart')
])

# Update month dropdown based on selected year
@app.callback(
    Output('month-dropdown', 'options'),
    Output('month-dropdown', 'value'),
    Input('year-dropdown', 'value')
)
def update_month_options(selected_year):
    months = uk[uk['Year'] == selected_year]['Month'].unique()
    options = [{'label': f'{m:02d}', 'value': m} for m in sorted(months)]
    return options, options[0]['value'] if options else None

# Update line chart based on year and month
@app.callback(
    Output('line-chart', 'figure'),
    Input('year-dropdown', 'value'),
    Input('month-dropdown', 'value')
)
def update_chart(year, month):
    filtered = uk[(uk['Year'] == year) & (uk['Month'] == month)]
    filtered['Date'] = pd.to_datetime(filtered[['Year', 'Month', 'Day']])
    df_grouped = filtered.groupby('Date')['Quantity'].sum().reset_index()

    fig = px.line(df_grouped, x='Date', y='Quantity',
                  title=f'Quantity Sold in {year}-{month:02d}')
    return fig

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [15]:
agg_func = {'InvoiceNo':'nunique',
            'Quantity':'sum',
            'UnitPrice':'mean',
            'Revenue':'sum',
            'CustomerID':'nunique'}

products = uk.groupby('Description').agg(agg_func).reset_index()

In [ ]:
app = dash.Dash(__name__)
app.title = "Product Analysis"

# Макет приложения
app.layout = html.Div([
    html.H1("Invoices vs Customers per Product"),
    
    dcc.Graph(id='scatter-plot'),
    
    html.Label("InvoiceNo Range:"),
    dcc.RangeSlider(
        id='x-slider',
        min=products['InvoiceNo'].min(),
        max=products['InvoiceNo'].max(),
        value=[products['InvoiceNo'].min(), products['InvoiceNo'].max()],
        marks=None,
        tooltip={"placement": "bottom", "always_visible": True}
    ),
    
    html.Label("CustomerID Range:"),
    dcc.RangeSlider(
        id='y-slider',
        min=products['CustomerID'].min(),
        max=products['CustomerID'].max(),
        value=[products['CustomerID'].min(), products['CustomerID'].max()],
        marks=None,
        tooltip={"placement": "bottom", "always_visible": True}
    ),
])

# Колбэк для обновления графика
@app.callback(
    Output('scatter-plot', 'figure'),
    Input('x-slider', 'value'),
    Input('y-slider', 'value')
)
def update_plot(x_range, y_range):
    filtered = products[
        (products['InvoiceNo'] >= x_range[0]) & (products['InvoiceNo'] <= x_range[1]) &
        (products['CustomerID'] >= y_range[0]) & (products['CustomerID'] <= y_range[1])
    ]
    
    fig = px.scatter(
        filtered,
        x='InvoiceNo',
        y='CustomerID',
        hover_name='Description',
        title='Products: Number of Invoices vs Number of Customers'
    )
    
    fig.update_traces(marker=dict(size=10, color='royalblue'))
    fig.update_layout(height=600)
    
    return fig

# Запуск
if __name__ == '__main__':
    app.run(debug=True)

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.

In [19]:

agg_data = data.groupby('Description')['Revenue'].sum().reset_index()

def plot_revenue_by_product(filter_text=""):
    # Filter the aggregated DataFrame: if filter_text is provided, filter case-insensitively;
    # otherwise, show all products.
    filtered = agg_data[agg_data['Description'].str.contains(filter_text, case=False, na=False)]
    
    # Create the bar chart: show revenue by product description.
    fig = px.bar(
        filtered, 
        x='Description', 
        y='Revenue',
        title=f"Revenue by Product Description (filter: '{filter_text}')",
        labels={'Revenue': 'Revenue', 'Description': 'Product Description'}
    )
    # Optionally, order categories by revenue.
    fig.update_layout(xaxis={'categoryorder': 'total descending'})
    fig.show()

# Create an interactive widget with a text field.
interact(plot_revenue_by_product, filter_text=widgets.Text(value="", description="Filter:"))


interactive(children=(Text(value='', description='Filter:'), Output()), _dom_classes=('widget-interact',))

<function __main__.plot_revenue_by_product(filter_text='')>